In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

from sklearn.cluster import KMeans

import missingno as msno

from fancyimpute import IterativeImputer as MICE
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam 


from sklearn.cluster import DBSCAN
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
from collections import Counter

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

from imblearn.over_sampling import KMeansSMOTE
from sklearn.mixture import GaussianMixture


from xgboost import XGBClassifier
from rgf.sklearn import RGFClassifier  # Regularized Greedy Forest
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from joblib import dump, load


In [2]:
df = pd.read_excel("C:\\Users\\dev\\Desktop\\Msc thesis Prior RS\\ML training\\df_mice_labeled_after_PCA.xlsx")

In [3]:
df.head()

,Unnamed: 0,Liquidity_and_Coverage_Ratios_PC1,Liquidity_and_Coverage_Ratios_PC2,Leverage_Ratios_PC1,Leverage_Ratios_PC2,Activity_Ratios_PC1,Activity_Ratios_PC2,Profitability_Ratios_PC1,Profitability_Ratios_PC2,Cost_and_Expense_Ratios_PC1,Cost_and_Expense_Ratios_PC2,Cash_Flow_Ratios_PC1,Cash_Flow_Ratios_PC2,Growth_Ratios_PC1,Growth_Ratios_PC2,Per_Share_Ratios_PC1,Per_Share_Ratios_PC2,LABEL
0,0,1.044059,-0.003853,-0.667224,-0.015571,-0.079080,-1.156702,0.182547,-0.106909,-0.030966,-0.108589,-0.466180,-0.944340,-0.035590,0.014286,-0.047160,0.006053,0
1,1,1.638262,-0.003264,-0.714215,-0.016845,-0.074370,-1.146165,0.150074,-0.121664,-0.028512,-0.083375,-0.210559,-0.361940,-0.057937,0.151235,-0.047160,0.006049,0
2,2,0.794693,-0.003885,-0.697014,-0.007657,-0.074604,-1.141965,0.138172,-0.106356,-0.029246,-0.087909,-0.460143,-1.000315,-0.040194,-0.040275,-0.047160,0.006054,0
3,3,0.730787,-0.004287,-0.617090,-0.012684,-0.080928,-1.119724,0.136261,-0.119001,-0.030920,-0.108539,-0.524498,-1.068359,-0.013969,-0.014758,-0.047161,0.006053,0
4,4,0.670656,-0.004352,-0.614995,-0.014103,-0.077759,-1.138776,0.177989,-0.108630,-0.031038,-0.109126,-0.330842,-0.244092,-0.037466,0.074644,-0.047157,0.006050,0


In [11]:
def split_dataset(dataset, target_column, test_size=0.2):
    X = dataset.drop(columns=[target_column])
    y = dataset[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    print("Dataset has been split and returned")
    print(" ")
    return X_train, X_test, y_train, y_test

def train_ann(X_train, y_train):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(12, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=0)

    print("ANN has been trained")
    print(" ")

    return model

def train_models(X_train, y_train):
    models = {}

    # Hyperparameter grid for each model
    param_grids = {
        'RandomForest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        },
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 6],
            'learning_rate': [0.01, 0.1]
        },
        'SVM': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        },
        'LogisticRegression': {
            'C': [0.1, 1, 10],
            'penalty': ['l2']
        },
        'GradientBoosting': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5, 7]
        },
        'KNN': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    }

    # ANN
    models['ANN'] = train_ann(X_train, y_train)

    print("ANN has been trained")
    print(" ")

    # Random Forest
    rf = RandomizedSearchCV(RandomForestClassifier(), param_grids['RandomForest'], cv=5, n_iter=8, random_state=42)
    rf.fit(X_train, y_train)
    models['RandomForest'] = rf.best_estimator_

    print("RandomForest has been trained")
    print(" ")

    # XGBoost
    xgb_model = RandomizedSearchCV(XGBClassifier(), param_grids['XGBoost'], cv=5, n_iter=8, random_state=42)
    xgb_model.fit(X_train, y_train)
    models['XGBoost'] = xgb_model.best_estimator_

    print("XGBoost has been trained")
    print(" ")

    # SVM
    svm = RandomizedSearchCV(SVC(probability=True), param_grids['SVM'], cv=5, n_iter=8, random_state=42)
    svm.fit(X_train, y_train)
    models['SVM'] = svm.best_estimator_

    print("SVM has been trained")
    print(" ")

    # Logistic Regression
    lr = RandomizedSearchCV(LogisticRegression(), param_grids['LogisticRegression'], cv=5, n_iter=8, random_state=42)
    lr.fit(X_train, y_train)
    models['LogisticRegression'] = lr.best_estimator_

    print("LogisticRegression has been trained")
    print(" ")

    # Gradient Boosting
    gb = RandomizedSearchCV(GradientBoostingClassifier(), param_grids['GradientBoosting'], cv=5, n_iter=8, random_state=42)
    gb.fit(X_train, y_train)
    models['GradientBoosting'] = gb.best_estimator_

    print("GradientBoosting has been trained")
    print(" ")

    # K-Nearest Neighbors
    knn = RandomizedSearchCV(KNeighborsClassifier(), param_grids['KNN'], cv=5, n_iter=8, random_state=42)
    knn.fit(X_train, y_train)
    models['KNN'] = knn.best_estimator_

    print("KNN has been trained")
    print(" ")

    # Naive Bayes
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    models['NaiveBayes'] = nb

    print("Naive Bayes has been trained")
    print(" ")

    return models

def test_models(models, X_test):
    predictions = {}
    for name, model in models.items():
        if name == 'ANN':
            predictions[name] = (model.predict(X_test) > 0.5).astype("int32")
        else:
            predictions[name] = model.predict(X_test)

    print("Models have been tested")
    print(" ")

    return predictions

def evaluate_models(models, predictions, y_test, X_test):
    metrics = {}
    for name, y_pred in predictions.items():
        accuracy = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, models[name].predict_proba(X_test)[:, 1]) if name != 'ANN' else roc_auc_score(y_test, models[name].predict(X_test))
        metrics[name] = {
            'accuracy': accuracy,
            'confusion_matrix': cm,
            'f1_score': f1,
            'auc_roc': auc
        }

    print("Models have been evaluated")
    print(" ")

    return metrics

def main(dataset, target_column):
    X_train, X_test, y_train, y_test = split_dataset(dataset, target_column)

    # Standardization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("Data has been standardized")
    print(" ")

    models = train_models(X_train, y_train)
    predictions = test_models(models, X_test)
    metrics = evaluate_models(models, predictions, y_test, X_test)

    # Save the models
    for name, model in models.items():
        if name != 'ANN':  # ANN model serialization handled differently
            dump(model, f'{name}_model.joblib')

    print("Models have been saved")
    print(" ")

    return metrics

def modelling(df):
    target_column = 'LABEL'  # Replace with your target column
    results = main(df, target_column)
    print(results)

In [12]:
modelling(df)

Dataset has been split and returned
 
Data has been standardized
 
ANN has been trained
 
RandomForest has been trained
 
XGBoost has been trained
 


c:\Users\dev\Desktop\MSC thesis\Code\mscthesis\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=8. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


SVM has been trained
 


c:\Users\dev\Desktop\MSC thesis\Code\mscthesis\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 3 is smaller than n_iter=8. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


LogisticRegression has been trained
 
GradientBoosting has been trained
 


c:\Users\dev\Desktop\MSC thesis\Code\mscthesis\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=8. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KNN has been trained
 
Naive Bayes has been trained
 
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
Models have been tested
 
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
Models have been evaluated
 
Models have been saved
 
{'ANN': {'accuracy': 0.9880745341614907, 'confusion_matrix': array([[1989,   23],
       [  25, 1988]], dtype=int64), 'f1_score': 0.9880715705765407, 'auc_roc': 0.9985641293816832}, 'RandomForest': {'accuracy': 0.9937888198757764, 'confusion_matrix': array([[2008,    4],
       [  21, 1992]], dtype=int64), 'f1_score': 0.9937640309304066, 'auc_roc': 0.9997846996510751}, 'XGBoost': {'accuracy': 0.995527950310559, 'confusion_matrix': array([[2005,    7],
       [  11, 2002]], dtype=int64), 'f1_score': 0.9955246146195923, 'auc_roc': 0.9998471663807518}, 'SVM': {'accuracy': 0.9836024844720497, 'confusion_matrix': array([[1986,   26],
       [  40, 1973]], dtype=int64), 'f1_score': 0.9835493519441675, 'auc_roc': 0.99730800492623}, 'LogisticRegression': {'accuracy': 0.97788819

In [16]:
# for using grid search cv

def split_dataset(dataset, target_column, test_size=0.2):
    X = dataset.drop(columns=[target_column])
    y = dataset[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    print("Dataset has been split and returned")
    print(" ")
    return X_train, X_test, y_train, y_test

def train_ann(X_train, y_train):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(12, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=0)

    print("ANN has been trained")
    print(" ")

    return model

def train_models(X_train, y_train):
    models = {}

    # Hyperparameter grid for each model
    param_grids = {
        'RandomForest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        },
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 6],
            'learning_rate': [0.01, 0.1]
        },
        'SVM': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        },
        'LogisticRegression': {
            'C': [0.1, 1, 10],
            'penalty': ['l2']
        },
        'GradientBoosting': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5, 7]
        },
        'KNN': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    }

    # ANN
    models['ANN'] = train_ann(X_train, y_train)

    # Random Forest
    rf = GridSearchCV(RandomForestClassifier(), param_grids['RandomForest'], cv=5)
    rf.fit(X_train, y_train)
    models['RandomForest'] = rf.best_estimator_

    print("RandomForest has been trained")
    print(" ")

    # XGBoost
    xgb_model = GridSearchCV(XGBClassifier(), param_grids['XGBoost'], cv=5)
    xgb_model.fit(X_train, y_train)
    models['XGBoost'] = xgb_model.best_estimator_

    print("XGBoost has been trained")
    print(" ")

    # SVM
    svm = GridSearchCV(SVC(probability=True), param_grids['SVM'], cv=5)
    svm.fit(X_train, y_train)
    models['SVM'] = svm.best_estimator_

    print("SVM has been trained")
    print(" ")

    # Logistic Regression
    lr = GridSearchCV(LogisticRegression(), param_grids['LogisticRegression'], cv=5)
    lr.fit(X_train, y_train)
    models['LogisticRegression'] = lr.best_estimator_

    print("LogisticRegression has been trained")
    print(" ")

    # Gradient Boosting
    gb = GridSearchCV(GradientBoostingClassifier(), param_grids['GradientBoosting'], cv=5)
    gb.fit(X_train, y_train)
    models['GradientBoosting'] = gb.best_estimator_

    print("GradientBoosting has been trained")
    print(" ")

    # K-Nearest Neighbors
    knn = GridSearchCV(KNeighborsClassifier(), param_grids['KNN'], cv=5)
    knn.fit(X_train, y_train)
    models['KNN'] = knn.best_estimator_

    print("KNN has been trained")
    print(" ")

    # Naive Bayes
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    models['NaiveBayes'] = nb

    print("Naive Bayes has been trained")
    print(" ")

    return models

def test_models(models, X_test):
    predictions = {}
    for name, model in models.items():
        if name == 'ANN':
            predictions[name] = (model.predict(X_test) > 0.5).astype("int32")
        else:
            predictions[name] = model.predict(X_test)

    print("Models have been tested")
    print(" ")

    return predictions

def evaluate_models(models, predictions, y_test, X_test):
    metrics = {}
    for name, y_pred in predictions.items():
        accuracy = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, models[name].predict_proba(X_test)[:, 1]) if name != 'ANN' else roc_auc_score(y_test, models[name].predict(X_test))
        metrics[name] = {
            'accuracy': accuracy,
            'confusion_matrix': cm,
            'f1_score': f1,
            'auc_roc': auc
        }

    print("Models have been evaluated")
    print(" ")

    return metrics

def main(dataset, target_column):
    X_train, X_test, y_train, y_test = split_dataset(dataset, target_column)

    # Standardization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("Data has been standardized")
    print(" ")

    models = train_models(X_train, y_train)
    predictions = test_models(models, X_test)
    metrics = evaluate_models(models, predictions, y_test, X_test)

    # Save the models
    for name, model in models.items():
        if name != 'ANN':  # ANN model serialization handled differently
            dump(model, f'{name}_model.joblib')

    print("Models have been saved")
    print(" ")

    return metrics

def modelling_gs(df):
    target_column = 'LABEL'  # Replace with your target column
    results = main(df, target_column)
    print(" ")
    print(results)

In [17]:
modelling_gs(df)

Dataset has been split and returned
 
Data has been standardized
 
ANN has been trained
 
RandomForest has been trained
 
XGBoost has been trained
 
SVM has been trained
 
LogisticRegression has been trained
 
GradientBoosting has been trained
 
KNN has been trained
 
Naive Bayes has been trained
 
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
Models have been tested
 
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step
Models have been evaluated
 
Models have been saved
 
{'ANN': {'accuracy': 0.9870807453416149, 'confusion_matrix': array([[1985,   27],
       [  25, 1988]], dtype=int64), 'f1_score': 0.9870903674280039, 'auc_roc': 0.999184722761296}, 'RandomForest': {'accuracy': 0.9932919254658386, 'confusion_matrix': array([[2008,    4],
       [  23, 1990]], dtype=int64), 'f1_score': 0.9932617918642376, 'auc_roc': 0.9997486516568745}, 'XGBoost': {'accuracy': 0.995527950310559, 'confusion_matrix': array([[2005,    7],
       [  11, 2002]], dtype=int64), 'f1_score': 0.9955246146195923, 'auc_roc': 